In [1]:
# Prereqs
import numpy as np
from hdf5storage import loadmat
import mne
from pathlib import Path

In [2]:
# mat = loadmat("/mnt/D/University/Fall 2025/BCI/Dataset/Task1_Preprocessed/ZAB/gip_ZAB_SNR6_EEG.mat")
mat = loadmat("/home/g4/Documents/BCI_Project/Dataset/Additional/Task3_Preprocessed/ZAB/gip_ZAB_TSR1_EEG.mat")
EEG = mat['EEG']

In [3]:
data = EEG['data']         # shape (n_channels, n_samples)
sfreq = float(EEG['srate'].item())   # sampling rate, e.g. 500.0
chanlocs = EEG['chanlocs']           # should be structured array

data.shape

(1, 105, 132033)

In [4]:
# Build ch_names and positions dict
ch_names = []
ch_pos = {}

# Extract labels and coordinates (X,Y,Z)
# NOTE: check how the structured object presents fields in your loader:
for i in range(len(chanlocs[0,0])):
    ch = chanlocs[0,0,i]            # or chanlocs[0,i] depending on shape
    name = str(ch['labels'].squeeze()).strip()            # might be array-like or object
    ch_names.append(name)
    # Coordinates:
    X = float(ch['X'].squeeze())   # inspect types
    Y = float(ch['Y'].squeeze())
    Z = float(ch['Z'].squeeze())
    # Convert to meters — you must verify units; here's a heuristic:
    # If coords look like 100-200, they might be mm -> divide by 1000
    # If coords look like 10-20, they might be cm -> divide by 100

    # Only add channels with valid coordinates
    if np.isfinite([X, Y, Z]).all() and not (X == 0 and Y == 0 and Z == 0):
        # Convert to meters - adjust divisor based on your data units
        ch_pos[name] = np.array([X, Y, Z]) / 100.0
    else:
        print(f"Warning: Channel {name} has invalid coordinates: X={X}, Y={Y}, Z={Z}")

print(f"Total channels: {len(ch_names)}")
print(f"Channels with valid positions: {len(ch_pos)}")

# Create Montage
montage = mne.channels.make_dig_montage(ch_pos=ch_pos, coord_frame='head')

Total channels: 105
Channels with valid positions: 105


In [5]:
# Typical head radius ~0.09 m. Quick check:
radii = [np.linalg.norm(pos) for pos in ch_pos.values()]
print("median head radius (m):", np.median(radii))
print("Min radius (m):", np.min(radii))
print("Max radius (m):", np.max(radii))
# If median >> 0.2 or << 0.03, you likely used wrong conversion factor.

median head radius (m): 0.08714432647690352
Min radius (m): 0.07295316373429835
Max radius (m): 0.09593040879824193


In [6]:
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
raw = mne.io.RawArray(data[0], info)
raw.set_montage(montage, match_case=False, on_missing="warn")
raw.set_eeg_reference('average', projection=True)

Creating RawArray with float64 data, n_channels=105, n_times=132033
    Range : 0 ... 132032 =      0.000 ...   264.064 secs
Ready.
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


<RawArray | 105 x 132033 (264.1 s), ~105.9 MiB, data loaded>

In [7]:
missing_channels = []
for ch in raw.info['chs']:
    loc3 = np.array(ch['loc'][:3])
    if not np.all(np.isfinite(loc3)) or np.linalg.norm(loc3) == 0:
        missing_channels.append(ch['ch_name'])

if missing_channels:
    print(f"\nDropping {len(missing_channels)} channels without valid 3D locations:")
    print(missing_channels)
    raw.drop_channels(missing_channels)
else:
    print("\nAll channels have valid 3D locations!")

info = raw.info


All channels have valid 3D locations!


In [8]:
# Use MNE sample data or fsaverage (download if necessary)
from mne.datasets import fetch_fsaverage
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = Path(fs_dir).parent.as_posix()  # path to subjects dir
subject = 'fsaverage'

# create source space (surface)
src = mne.setup_source_space(subject, spacing='oct6', subjects_dir=subjects_dir, add_dist=False)

# create BEM model (3-layer) using sample surfaces shipped with MNE
# MNE provides precomputed fsaverage BEM; if not available compute one with mne.make_bem_model
bem = mne.read_bem_solution(Path(subjects_dir) / subject / 'bem' / f'{subject}-5120-5120-5120-bem-sol.fif')

# If you don't have BEM, you can create a 3-layer sphere model (simpler but less accurate)
# bem_sphere = mne.make_sphere_model(r0=(0., 0., 0.), head_radius=0.095)

# compute forward solution (leading field)
trans = None  # no coreg transform; using montage + fsaverage assume alignment (not ideal)
fwd = mne.make_forward_solution(info, trans=trans, src=src, bem=bem, meg=False, eeg=True, mindist=5.0, verbose=True)

0 files missing from root.txt in /home/g4/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/g4/mne_data/MNE-fsaverage-data/fsaverage
Setting up the source space with the following parameters:

SUBJECTS_DIR = /home/g4/mne_data/MNE-fsaverage-data
Subject      = fsaverage
Surface      = white
Octahedron subdivision grade 6

>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading /home/g4/mne_data/MNE-fsaverage-data/fsaverage/surf/lh.white...
Mapping lh fsaverage -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from /home/g4/mne_data/MNE-fsaverage-data/fsaverage/surf/lh.sphere...
Setting up the triangulation for the decimated surface...
loaded lh.white 4098/163842 selected to source space (oct = 6)

Loading /home/g4/mne_data/MNE-fsaverage-data/fsaverage/surf/rh.white...
Mapping rh fsaverage -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from /home/g4/mne_data/MNE-fsaverage-data/fsaverage/s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s finished


    Found    0/3192 points outside using solid angles
    Total 4098/4098 points inside the surface
Interior check completed in 1946.0 ms

Setting up for EEG...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s finished


Computing EEG at 8196 source locations (free orientations)...

Finished.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s finished


In [9]:
# Example: create noise covariance from first N seconds of recording
baseline_seconds = 10
# baseline_samples = int(baseline_seconds * sfreq)
cov = mne.compute_raw_covariance(raw, tmin=0.0, tmax=baseline_seconds)  # e.g. first 10s

Using up to 50 segments
Number of samples used : 5000
[done]


In [10]:
from mne.minimum_norm import make_inverse_operator, apply_inverse_raw

inv_operator = make_inverse_operator(info, fwd, cov, loose=0.2, depth=0.8)

# To apply to an evoked (word-locked) object:
# evoked = data[0]
stc = apply_inverse_raw(raw, inv_operator, lambda2=1.0/9.0, method='dSPM', start=0, stop=int(60 * sfreq), buffer_size=1000)  # method: 'MNE','dSPM','sLORETA'
stc

Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 105 channels.
    105 out of 105 channels remain after picking
Selected 105 channels
Creating the depth weighting matrix...
    105 EEG channels
    limit = 8197/8196 = 2.357499
    scale = 180302 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 0.31 (2.2e-16 eps * 105 dim * 1.3e+13  max singular value)
    Estimated rank (eeg): 22
    EEG: rank 22 computed from 105 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing S

/tmp/ipykernel_9677/933817925.py:3: RuntimeWarning: The largest eigenvalue of the 105-channel eeg covariance (rank=22) is over 10 orders of magnitude larger than the smallest (1.35e+03 > 1e10 * 8.24e-11), the resulting whitener will likely be unstable
  inv_operator = make_inverse_operator(info, fwd, cov, loose=0.2, depth=0.8)
/tmp/ipykernel_9677/933817925.py:3: RuntimeWarning: The largest eigenvalue of the 105-channel eeg covariance (rank=22) is over 10 orders of magnitude larger than the smallest (1.35e+03 > 1e10 * 8.24e-11), the resulting whitener will likely be unstable
  inv_operator = make_inverse_operator(info, fwd, cov, loose=0.2, depth=0.8)


    largest singular value = 4.44452
    scaling factor to adjust the trace = 2.16554e+16 (nchan = 105 nzero = 83)
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 22 (83 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Applying inverse to raw...
    Picked 105 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    computing inverse and combining the current components (using 30 segments)...
        segment 1 / 30 done..
        segment 2 / 30 done..
        segment 3 / 30 done..
        segment 4 / 30 done..
        segment 5 / 30 done..
        segment 6 / 30 done..
        segment 7 / 30 done..
        segment 8 / 30 done..
        segment 9 / 30 done..
        segment 10 / 30 done..
        segment 

<SourceEstimate | 8196 vertices, subject : fsaverage, tmin : 0.0 (ms), tmax : 59998.00000000001 (ms), tstep : 2.0 (ms), data shape : (8196, 30000), ~1.83 GiB>

In [11]:
stc.data.shape

(8196, 30000)

In [13]:
from mne.minimum_norm import write_inverse_operator

info.save("info.fif")
mne.write_forward_solution('forward_solution-fwd.fif', fwd, overwrite=True)
write_inverse_operator('inverse_operator-inv.fif', inv_operator, overwrite=True)

    Write a source space...


    [done]
    Write a source space...
    [done]
    2 source spaces written
Write inverse operator decomposition in /home/g4/Documents/BCI_Project/BCI_Project/inverse_operator-inv.fif...
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Writing inverse operator info...
    Writing noise covariance matrix.
    Writing source covariance matrix.
    Writing orientation priors.
    [done]


In [ ]:
from mne.beamformer import make_lcmv, apply_lcmv
# compute data covariance in time window of interest (e.g. word epoch)
data_cov = mne.compute_covariance(epochs, tmin=0.0, tmax=None)
filters = make_lcmv(info, fwd, data_cov, reg=0.05, noise_cov=cov, pick_ori='max-power')
stc = apply_lcmv(epochs.average(), filters)   # yields source estimate